In [3]:
import morePVs_output as opm
import morePVs as mpv
import os
import en_utilities as um
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

Plotting Total Building Costs / %bau vs kWp and kWh storage
----------------------------------------------------------------
Site J
Adapted from 'EN1_value4 en pv - LINEPLOTS - 3 with a_term - % bau' (which has no storage
next version: plotting colour (%$) vs kWp and kWh

study_`siteJ_bat2`
Single set of parametetrs:
* `en_pv`
* `EA310_10c`
* No `FiT`
* `capex_5`
* `a_term` = 12 years
* `powerwall2` x 1, 2 or 3
* `evening_discharge1`

__Need to expand this to range of parameters, and for other sites__
 


In [4]:
# -------------------------------
# combine  input and result files
# -------------------------------
# Change to bat2
project='p_testing' #'EN1_pv_bat1'
study_name='test_pv_storage' #'combined_siteJ_bat4'
base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\studies'
op = opm.Output(base_path = base_path,
                          project = project,
                          study_name = study_name)
self = op
df = self.data.copy()
df_in = self.study_parameters.copy()
df = df.merge(df_in,left_index = True, right_index=True)
for c in df.columns:
    if '_x' in c:
        nc = c[:-2]
        df.rename(columns={c:nc}, inplace=True)
    if '_y' in c:
        df= df.drop([c],axis=1)
df.head()



,scenario_label,arrangement,number_of_households,load_folder,central_battery_kWh,en_opex,en_capex_repayment,pv_capex_repayment,average_hh_bill$,average_hh_total$,...,pv_cap_id,cp,all_residents,parent,network_tariff,en_capex_id,a_term,a_rate,battery_id,battery_strategy
scenario,,,,,,,,,,,,,,,,,,,,,
0,test_pv_storage_000,bau,26.0,h_one_site_J,0.0,0.0,0.000000,0.000000,1171.782103,1171.782103,...,NaN,EASO_TOU_15pc_FIT12,EASO_TOU_15pc_FIT12,TIDNULL,EA310,capex_null,20,0.06,NaN,NaN
1,test_pv_storage_001,en,26.0,h_one_site_J,0.0,6500.0,2613.540501,0.000000,0.000000,0.000000,...,NaN,TIDNULL,TIDNULL,EA310_12c,EA310,capex_med,20,0.06,NaN,NaN
2,test_pv_storage_002,en_pv,26.0,h_one_site_J,0.0,6500.0,2613.540501,9380.804994,0.000000,0.000000,...,site_J_max,TIDNULL,TIDNULL,EA310_12c,EA310,capex_med,20,0.06,NaN,NaN
3,test_pv_storage_003,en_pv,26.0,h_one_site_J,0.0,6500.0,2613.540501,7823.427159,0.000000,0.000000,...,site_J_2_5kw,TIDNULL,TIDNULL,EA310_12c,EA310,capex_med,20,0.06,NaN,NaN
4,test_pv_storage_004,en_pv,26.0,h_one_site_J,0.0,6500.0,2613.540501,6258.741727,0.000000,0.000000,...,site_J_2_0kw,TIDNULL,TIDNULL,EA310_12c,EA310,capex_med,20,0.06,NaN,NaN


In [5]:
study_name

'test_pv_storage'

In [6]:
# Directories for plots
# ---------------------
plot_bau = os.path.join(op.plot_path,'percentage_bau')
if not os.path.exists(plot_bau):
    os.makedirs(plot_bau)
plot_en = os.path.join(op.plot_path,'percentage_en')
if not os.path.exists(plot_en):
    os.makedirs(plot_en)

In [7]:
# ---------------
# Organise labels
# ---------------
df.loc[:,'site'] = df.loc[:,'load_folder'].apply(lambda x : x[-1])
df.loc[:, 'labels'] = df.loc[:,'site']
floors = {'A':12,'E':7, 'D':9,'B':8,'H':3,'I':4,'G':4,'C':4,'J':4,'F':5}
numhouses={'A':208,'E':161, 'D':138,'B':104,'H':52,'I':48,'G':44,'C':34,'J':26,'F':20}
labels ={}
for s, f in floors.items():
    u = numhouses[s]
    labels[s] = s +'('+str(u)+'/'+str(f)+')'
print(labels)
df.loc[:, 'label'] = df.loc[:, 'site'].apply(lambda x: labels[x])
# ----------------
# Set up constants
# ----------------
# NB Some of these are now superfluous
sites=['A','E','D','B','H','I','G','C','J','F']
# Plotting set-up parameters
cmap = mpl.cm.seismic
alpha=1

markers =['x','o','D','*','^','H','s','v','>','<']
site_markers = dict(zip(sites,markers))

colours = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
site_colours = dict(zip(sites,colours))

a_terms =df.a_term.tolist()
term_markers = dict(zip(a_terms,markers))
# dashList = [(5,2),(2,5),(4,10),(3,3,2,2),(5,2,20,2)] 
# term_dashes = dict(zip(a_terms,dashList))

# get pv sizes for maximum pv systems
# -----------------------------------
maximums={}
pv_ref_file='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\reference\\capex_pv_lookup.csv'
pv_ref=pd.read_csv(pv_ref_file)
pv_ref = pv_ref.set_index('pv_cap_id')
for i in pv_ref.index:
    if 'max' in i and 'site' in i:
        site = um.find_between(i,'_','_')
        maximums[site] = pv_ref.loc[i,'kW']
# ----------------------------------
# get lists of parameters to iterate
# ----------------------------------
en_capexes= [c for c in ['capex_4','capex_5','capex_6']if c in df.en_capex_id.tolist()]
# parents = [p for p in ['EA310_12c_FIT8', 'EA310_12c_FIT12', 'EA310_12c',\
#            'EA310_10c_FIT8', 'EA310_10c_FIT12', 'EA310_10c',\
#          'EA310_8c_FIT8', 'EA310_8c_FIT12','EA310_8c'] if p in df.parent.tolist()]
parents =df.parent.drop_duplicates().tolist()

{'A': 'A(208/12)', 'E': 'E(161/7)', 'D': 'D(138/9)', 'B': 'B(104/8)', 'H': 'H(52/3)', 'I': 'I(48/4)', 'G': 'G(44/4)', 'C': 'C(34/4)', 'J': 'J(26/4)', 'F': 'F(20/5)'}


In [8]:
dfb = df[df['arrangement']=='bau']
baulist = dfb.index.tolist()
dfb = dfb.drop_duplicates('arrangement')
df = df.drop(baulist).append(dfb)


In [9]:
# get kWp / unit , total $ / unit  and % bau for each scenario
# ------------------------------------------------------------
df['pv_filename'].fillna('zero',inplace=True)

for s in df.index:
    site = df.loc[s,'load_folder'][-1]
    df.loc[s,'site']=site
    if df.loc[s,'pv_filename'] == 'zero':
        df.loc[s,'kWp_unit'] =0
    elif 'max' in df.loc[s,'pv_filename']:
        df.loc[s,'kWp_unit']= maximums[site]/df.loc[s,'number_of_households']
    else:
        df.loc[s,'kWp_unit'] = float(df.loc[s,'pv_filename'][-9]) + float(df.loc[s,'pv_filename'][-7])/10
    df.loc[s,'$_unit'] = df.loc[s,'total$_building_costs_mean']/df.loc[s,'number_of_households']
    bau_scenario = df[(df['site']==site) & (df['arrangement'] == 'bau')].index
    df.loc[s,'bau'] = (df.loc[bau_scenario,'total$_building_costs_mean']).values[0]/df.loc[s,'number_of_households']
    df.loc[s,'%_bau'] = df.loc[s,'$_unit']/ df.loc[s,'bau']*100
    df.loc[s,'kWh_unit']= df.loc[s,'central_battery_kWh']/df.loc[s,'number_of_households']
    if df.loc[s,'arrangement'] != 'bau':
        en_scenario = df[(df['site']==site) & (df['arrangement'] == 'en') & 
                         (df['parent'] == df.loc[s,'parent']) &  #(df['a_term'] == df.loc[s,'a_term']) &
                         (df['en_capex_id'] == df.loc[s,'en_capex_id'])].index

        df.loc[s,'en'] = (df.loc[en_scenario,'total$_building_costs_mean']).values[0]/df.loc[s,'number_of_households']
        df.loc[s,'%_en'] = df.loc[s,'$_unit']/ df.loc[s,'en']*100


In [10]:
# ---------------------
# prep and plot - % bau
# ---------------------
# Iterate through parent, capex, a_term combinations
for parent, capex, a_term in list(set([(df.loc[s,'parent'],df.loc[s,'en_capex_id'],df.loc[s,'a_term']) for s in df.index])):

    title = parent + " / " + capex                        
    p_slice = df['parent'] == parent 
    c_slice = df['en_capex_id'] == capex
    a_slice = df['a_term'] ==a_term
    dfx=df.copy()[p_slice & c_slice & a_slice]

    # Get rid of duplicates - assumed irrelevant as only variation is internal tariffs
    dfx.loc[:,'kWp_str'] = dfx.loc[:,'kWp_unit'].apply(lambda x : str(round(x,2)))
    dfx.loc[:,'aterm_str'] = dfx.loc[:,'a_term'].apply(lambda x : str(round(x,0)))
    dfx.loc[:,'kWh_str'] = dfx.loc[:,'kWh_unit'].apply(lambda x : str(round(x,2)))

    dfx['combined'] = dfx['site']+ '_' + dfx['arrangement'] + '_'+ dfx['kWp_str']+ '_'+(dfx['aterm_str']) + '_' + dfx.loc[:,'kWh_str']
    #dfx = dfx.drop_duplicates('combined')
    dfx['kWp_kWh'] =  dfx['kWp_str'] + '_'+ dfx['kWh_str']
    dfx['new_label']= dfx['label']+ '_'+ dfx['kWp_str'] + '_'+ dfx['aterm_str']

    # ----------------------
    # reindex and stack data
    # ----------------------
    dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
    dfu = dfx['%_bau'].unstack()
    cols =dfu.columns
    dfu.columns=cols
    dfv=dfu.transpose()
    # NB - parent, capex combinations must have sufficient data
    if (dfv.shape[0] <= 1 or dfv.shape[1] <= 1):
        continue
    
    # ---------------------
    # Plot % bau vs kW peak
    # ---------------------
    dfplot = dfv.copy()
    cols = dfplot.columns
    cols2 = [str(round(c,1)) + ' kWh / unit' for c in cols]
    dfplot.columns = cols2
    fig, ax = plt.subplots()
    # Legend outside box and resize chart to fit
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
    dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend (loc='upper left',bbox_to_anchor=(1.02, 1))
    maxy= dfplot.max().max()
    ax.set_ylabel('Total Cost as % of BAU')
    ax.set_xlabel('PV kWp / unit')
    pt = "Parent tariff = "+ parent
    et ="EN Capex = "+capex
    at = "A. Term = "+str(a_term) + "years"
    ax.text(0,maxy*0.98,pt)
    ax.text(0,maxy*0.96,et)
    ax.text(0,maxy*0.94,at)
    pname = 'bau_kWp'  +'_'+str(a_term) + '_'+ parent+'_' + capex + '.png'
    plotFile = os.path.join(plot_bau,pname)
    plt.savefig(plotFile,dpi=1000)
    plt.close()
    
    # -----------------
    # Plot % bau vs kWh 
    # -----------------
    dfplot = dfv.copy().T
    cols = dfplot.columns
    cols2 = [str(round(c,1)) + ' kWp / unit' for c in cols]
    dfplot.columns = cols2
    fig, ax = plt.subplots()
    # Legend outside box and resize chart to fit
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
    dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc='upper left',bbox_to_anchor=(1.02, 1))
    maxy= dfplot.max().max()
    ax.set_ylabel('Total Cost as % of BAU')
    ax.set_xlabel('Storage kWh / unit')
    pt = "Parent tariff = "+ parent
    et ="EN Capex = "+capex
    at = "A. Term = "+str(a_term) + "years"
    ax.text(0,maxy*0.98,pt)
    ax.text(0,maxy*0.96,et)
    ax.text(0,maxy*0.94,at)
    pname = 'bau_kWh'  +'_'+str(a_term) + '_'+ parent+'_' + capex + '.png'
    plotFile = os.path.join(plot_bau,pname)
    plt.savefig(plotFile,dpi=1000)
    plt.close()

In [11]:
# ---------------------
# prep and plot - % en
# ---------------------
# Iterate through parent, capex, a_term combinations
for parent, capex, a_term in list(set([(df.loc[s,'parent'],df.loc[s,'en_capex_id'],df.loc[s,'a_term']) for s in df.index])):
    title = parent + " / " + capex                        
    p_slice = df['parent'] == parent 
    c_slice = df['en_capex_id'] == capex
    a_slice = df['a_term'] == a_term
    dfx=df.copy()[p_slice & c_slice & a_slice]

    # Get rid of duplicates - assumed irrelevant as only variation is internal tariffs
    dfx.loc[:,'kWp_str'] = dfx.loc[:,'kWp_unit'].apply(lambda x : str(round(x,2)))
    dfx.loc[:,'aterm_str'] = dfx.loc[:,'a_term'].apply(lambda x : str(round(x,0)))
    dfx.loc[:,'kWh_str'] = dfx.loc[:,'kWh_unit'].apply(lambda x : str(round(x,2)))

    dfx['combined'] = dfx['site']+ '_' + dfx['arrangement'] + '_'+ dfx['kWp_str']+ '_'+(dfx['aterm_str']) + '_' + dfx.loc[:,'kWh_str']
    #dfx = dfx.drop_duplicates('combined')
    dfx['kWp_kWh'] =  dfx['kWp_str'] + '_'+ dfx['kWh_str']
    dfx['new_label']= dfx['label']+ '_'+ dfx['kWp_str'] + '_'+ dfx['aterm_str']


    # ----------------------
    # reindex and stack data
    # ----------------------
    dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
    dfu = dfx['%_en'].unstack()
    cols =dfu.columns

    dfu.columns=cols
    dfv=dfu.transpose()
    # NB - parent, capex combinations must have sufficient data
    if (dfv.shape[0] <= 1 or dfv.shape[1] <= 1):
        continue
    
    # --------------------
    # Plot % en vs kW peak
    # --------------------
    dfplot = dfv.copy()
    cols = dfplot.columns
    cols2 = [str(round(c,1)) + ' kWh / unit' for c in cols]
    dfplot.columns = cols2
    fig, ax = plt.subplots()
    # Legend outside box and resize chart to fit
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
    dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc='upper left',bbox_to_anchor=(1.02, 1))
    maxy= dfplot.max().max()
    ax.set_ylabel('Total Cost as % of EN (no PV)')
    ax.set_xlabel('PV kWp / unit')
    pt = "Parent tariff = "+ parent
    et ="EN Capex = "+capex
    at = "A. Term = "+str(a_term) + "years"
    ax.text(0,maxy*0.98,pt)
    ax.text(0,maxy*0.96,et)
    ax.text(0,maxy*0.94,at)
    pname = 'en_kWp'  +'_'+str(a_term) + '_'+ parent+'_' + capex + '.png'
    plotFile = os.path.join(plot_en,pname)
    plt.savefig(plotFile,dpi=1000)
    plt.close()
    
        # ----------------
    # Plot % EN vs kWh
    # ----------------
    dfplot = dfv.copy().T
    cols = dfplot.columns
    cols2 = [str(round(c,1)) + ' kWp / unit' for c in cols]
    dfplot.columns = cols2
    fig, ax = plt.subplots()
    # Legend outside box and resize chart to fit
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
    dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc='upper left',bbox_to_anchor=(1.02, 1))
    maxy= dfplot.max().max()
    ax.set_ylabel('Total Cost as % of EN (no PV)')
    ax.set_xlabel('Storage kWh / unit')
    pt = "Parent tariff = "+ parent
    et ="EN Capex = "+capex
    at = "A. Term = "+str(a_term) + "years"
    ax.text(0,maxy*0.98,pt)
    ax.text(0,maxy*0.96,et)
    ax.text(0,maxy*0.94,at)
    pname = 'en_kWh'  +'_'+str(a_term) + '_'+ parent+'_' + capex + '.png'
    plotFile = os.path.join(plot_en,pname)
    plt.savefig(plotFile,dpi=1000)
    plt.close()
